In [31]:
import numpy as np
import math
from scipy.fftpack import fft, ifft
import librosa as lb
from scipy.signal import get_window
import scipy.io
import scipy.fftpack as sc
import scipy.io.wavfile as scread
import matplotlib.pyplot as plt
import os
import re
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [32]:
def audioToVector(file):
    x,fs=lb.core.load(file,sr=44100)
    #Here FFT size is required to be same as sampling rate, otherwise
    #the frequency values also getting halved(peak values)
    
    y= sc.fft(x,fs)
    #taking abstract values(upper part of spectrum)
    y=abs(y)
    
    #Shifting the values to 440Hz range
    shift_index=440-np.argmax(y)
    y=np.roll(y[:fs//2],shift_index)
    
    #taking averga eof 100 values
    avgy=[]
    for i in range(50):
        avgy.append(np.mean(y[i*20:(i+1)*20]))
    return avgy
    

In [33]:
def vectorToDataset(custompath = ''):
    X=[]                                #Using lists instead of numpy for fast processing intially
    Y=[]
    print('\nRunning vectorToDataset Function...\n')
    print('We\'re here:',os.getcwd())                  #You should be just outside the 'Dataset' 
                                        #folder as this function runs in main working directory of project
    if custompath != '':
        os.chdir(path=custompath)       #Will be used when importing this function
        
    try:
        os.chdir(path=r'Dataset')
    except:
        pass
    ld=sorted(os.listdir())             #List of Instruments Folders
    print('\nInstrument Folders :',ld)
    r=re.compile(r'.*_.*4_.*')          #Regex to get notes from 4th octave only(normal and sharp notes)

    for i in range(len(ld)):
        os.chdir(path=ld[i])            #Change DIR to the instrument folder from 'ld'
        li = os.listdir()               #List of a different notes of a particular instrument 
        li=list(filter(r.match,li))     #Match Regex here
        print('\nCreating dataset for :',ld[i],'with',len(li),'Instruments')
                                        #Prints number of files found for an instrument to be used
        for j in li:
            file = os.getcwd() + '/' + j#gets the full path to the file
            x = audioToVector(file)     #Passes it for further processing to get the (50,1) vector
            Y.append(i)                 #Appends the label 'ld[i]' or its index 'i' 
            X.append(x)

        os.chdir(path=r'../')           #Gets back to Dataset folder

    X=np.array(X)                       #changes the list into numpy array(2D)
    Y=np.array(Y)[:,np.newaxis]         #changes the list into numpy array(2D)

    print('\nX.shape and Y.shape:',X.shape,Y.shape)
    pd.DataFrame(X).to_csv("../features.csv", index = None, header = None)
                                        #Save to .csv file
    pd.DataFrame(Y).to_csv("../lables.csv", index = None, header = None)
    

In [36]:
def test(default=False):
    if default==True:
        print('''
Testing the audioToVector Function:

        ''')
        t=audioToVector('violin/violin_A4_05_pianissimo_arco-normal.mp3')
        print(t,len(t))

        print('''
Testing the vectorToDataset Function:

        ''')
        Xdata=pd.read_csv("../features.csv",header = None)
                                            #Loading  values from the created dataset
        print('Size of Features:',Xdata.shape,type(Xdata),'\n')
        X=Xdata.values[:,:]                #Getting Numpy array from Dataframe
        print(X[:2])


if __name__ == "__main__":
    try:
        vectorToDataset()               #Create the Dataset
        test(True)                          #Default test mode = False
    except:
        print('\nSomething went wrong, Please check...\n')


Testing the audioToVector Function:

        
[4.3911557, 6.1085515, 1.5787694, 0.7517971, 0.7000272, 0.46272534, 0.2586546, 0.1952292, 0.13665023, 0.15581015, 0.13102049, 0.1601045, 0.13944599, 0.5773125, 0.39006907, 0.1940441, 0.20739011, 0.3312415, 0.3581831, 0.23548348, 1.0733955, 13.470314, 11.622615, 0.6320586, 0.20620081, 0.31046134, 0.10221362, 0.071646646, 0.082974, 0.10344581, 0.1243671, 0.102536455, 0.08502493, 0.07817264, 0.049993597, 0.07296754, 0.09024755, 0.106712654, 0.05051279, 0.12942415, 0.07469422, 0.0933516, 0.17465246, 1.9273554, 11.392414, 0.24876389, 0.13240206, 0.07995466, 0.09219417, 0.15543318] 50

Testing the vectorToDataset Function:

        
Size of Features: (1520, 50) <class 'pandas.core.frame.DataFrame'> 

[[  3.6565442    4.9872212   11.04625     16.581758    67.85458
    8.491892     2.1045792    1.2747738    1.9992796    1.8267685
    2.0068161    1.9434105    3.0274942    3.125929     3.8833618
    2.8523576    4.2764425    7.8642907    8.645449  